# Python语言的高级特性
---

## 1 面向对象程序设计

### 1.1 面向过程 vs. 面向对象

#### 面向过程程序设计（Procedure Oriented Programming, POP）：程序 = 算法 + 数据结构

- 面向过程的开发范式，是把程序划分为两个相互分离的部分：数据表示（即数据结构）和数据操作（即算法）
- POP的核心侧重于数据结构和算法的开发与优化
- 强调：程序的易读性
- 不足：无法应对需求变化。如果需求发生变化，代码的改动就会很大。这样对软件的后期维护和扩展不利。

#### 面向对象程序设计（Object OrientedProgramming，OOP）：程序 = 对象 + 消息传递

- 对象 = 数据 + 方法（操作）：每个对象中既包括数据，又包括对数据的处理
- 封装：每个对象都像一个封闭的小型机器，彼此协作，又不相互干扰
- 消息：对象之间通过“消息”进行通信，
- 面向对象设计使程序更容易扩展，也更加符合现实世界的模型
- 类（class)：将具有的相同属性（数据）及相同行为（对数据的操作）封装在一起，便创造了新的类，大大扩充了数据类型的概念
- 面向对象程序设计的重点是对类的设计，而非对象的实现。

### 1.2 类的定义与使用

In [94]:
class Person:
    #定义类的数据成员
    height = 140 
    # 构造方法
    def __init__(self, name, age, weight):
        # 定义对象的数据成员属性
        self.name = name 
        self.age = age 
        
        # 定义私有属性，私有属性在类外无法直接访问
        self.__weight = weight
        
    def speak(self):
        print('{} 说：我{}岁，体重为{}kg，身高为{}cm'
              .format(self.name, self.age, self.__weight, Person.height))

In [95]:
p1 = Person('Alice', 10, 30)
p1.speak()

Alice 说：我10岁，体重为30kg，身高为140cm


In [96]:
p1.age = 12
p1.name = 'Bob'
p1.speak()

Bob 说：我12岁，体重为30kg，身高为140cm


一般而言，在面向对象编程中，函数和方法可看作同义词。但在Python中，函数和方法还是有所不同的。方法是指与特定实例绑定的函数，因此，我们常把类中的函数称为方法（这一点类似于Java），而把不与实例绑定的普通功能块称为函数（如全局的内置函数print()、len()等）。

当通过对象调用方法时，对象本身（即self）将作为第一个参数被传递过去，而普通函数则不具备这个特性。

In [97]:
p2 = Person('Luna', 11, 31)
Person.height = 150
p1.speak()
p2.speak()

Bob 说：我12岁，体重为30kg，身高为150cm
Luna 说：我11岁，体重为31kg，身高为150cm


可以看出，更改了属于类的公有数据成员height的值，而该数据为所有对象共享，因此对象p1和p2中的height值都改变了。

而Python允许为对象添加新的临时属性。比如说，在上述代码中，p1对象没有属性nickname，但可以给它添加一个：

In [98]:
p1.nickname = '小李子'
p1.nickname

'小李子'

In [100]:
# p2.nickname # Error

### 1.3 类的继承

话说，“虎父无犬子”，“龙生龙，凤生凤，老鼠的儿子会打洞”，这在一定程度上说明了继承的重要性—优秀的特性要留给后辈。

在面向对象程序设计中，继承（Inheritance）是软件复用的关键技术。通过继承，子类可以复用父类的优秀特性，同时还可进一步扩充新的特性，适应新的需求。

在已有类的基础上新增自己的特性，继而产生新类的过程，称为派生。我们把既有的类称为基类（Base Class）、超类（Super Class）或者父类（Parent Class），而将派生出的新类称为派生类（Derived Class）或子类（Subclass）。

继承的目的在于实现代码重用，即对于已有的、成熟的功能，令子类从父类处奉行“拿来主义”。而派生的目的则在于，当新的问题出现且原有代码无法解决（或不能完全解决）时，需要对原有代码进行全部（或部分）改造。

对于面向对象的程序而言，设计孤立的类是比较容易的，难的是正确设计好类的层次结构以达到代码高效重用的目的。

In [109]:
class Student(Person):
    grad = None # 年级
    
    def __init__(self, name, age, weight, grad):
        # 调用父类的构造方法，初始化父辈数据成员
        Person.__init__(self, name, age, weight)
        self.grade = grad
    # 覆盖父类的同名方法   
    def speak(self):
        print('{} 说：我{}岁了，体重{}公斤。'.format(self.name, self.age, self._Person__weight), end="")
        if self.grade != None:
            print('我在读{}年级。'.format(self.grade))

In [110]:
st = Student('Alice', 11, 40, 5)
st.speak()

Alice 说：我11岁了，体重40公斤。我在读5年级。


## 2 生成器与迭代器

### 2.1 生成器

通过推导式，可以直接创建一个列表、字典或集合。但是，由于受到内存的限制，这些可迭代对象（列表、字典或集合）的容量是有限的。

根据局部性原理，在一段时间内我们要访问的仅仅局限于相邻的若干个元素，即使把所有元素都加载到内存之中，它们被访问的概率也非常小。因此，大部分的存储空间其实是被白白浪费了。

基于此，就会有这样的需求：这些元素能不能按照某种算法推算出来，然后在后续循环过程中，根据这些元素不断推算出其他被访问的元素呢？这样一来，就不必创建完整的列表、字典或集合了，从而节省了大量的空间。在Python语言中，这种一边循环一边计算的机制，称为生成器。

#### 使用生成器表达式

创建一个生成器并不复杂，方法也有很多。最简单的一种是把一个列表推导式最外层的标记方括号`[]`改成圆括号`()`，这样一个生成器就创建好了。

In [111]:
n = 10
# 列表推导式
a = [x**2 for x in range(n) if x % 2 == 0]
a

[0, 4, 16, 36, 64]

In [112]:
type(a)

list

In [113]:
# 生成器表达式
b = (x**2 for x in range(n) if x % 2 == 0)
b

<generator object <genexpr> at 0x7fbca8396b50>

In [114]:
type(b)

generator

In [115]:
# 通过生成器获取（下一个生成的）元素
next(b)

0

In [116]:
next(b)

4

In [117]:
next(b)

16

In [118]:
next(b)

36

In [119]:
next(b)

64

In [121]:
# next(b) #没有了，触发异常

In [124]:
fib_gen = (fibonacci(i) for i in range(10))
fib_gen

<generator object <genexpr> at 0x7fbca8354450>

In [125]:
for n in fib_gen:
    print(n, end=' ')

1 1 2 3 5 8 13 21 34 55 

#### 使用yield创建生成器

生成器的功能很强大。如果推算的算法比较复杂，难以利用列表推导式来生成，这时就可以使用含有yield关键字的函数。

In [126]:
# 斐波那契数列生成器
def fibonacci(n):
    i, a, b = 0, 0, 1
    while i < n:
        # 生成数列的第i项
        yield b
        # 计算下一项
        a, b = b, a + b
        i += 1
    return 'Done'

如果某个函数定义中包含yield关键字，那么这个函数就不一般了，它不再是一个普通函数，而是一个生成器。

In [128]:
fib = fibonacci(10)
fib

<generator object fibonacci at 0x7fbca83544d0>

In [129]:
next(fib)

1

In [130]:
next(fib)

1

In [131]:
next(fib)

2

In [132]:
fib.__next__()

3

In [133]:
next(fib)

5

In [135]:
fib = fibonacci(10)

In [136]:
for item in fib:
    print(item, end=' ')

1 1 2 3 5 8 13 21 34 55 

#### 生成器的执行流程

生成器和函数的执行流程不一样。普通函数遇到return语句或者执行到最后一行函数语句时就会返回，结束整个函数的运行。而变成生成器的函数，在每次调用next()的时候执行，遇到yield语句就“半途而废”，再次执行时，就会从上次返回的yield语句处接着往下执行。

In [25]:
# 简单生成器
def simple_gen():
    for i in range(10):
        print('simple_gen:','第{}次生成并返回{}'.format(i, i*i))
        yield(i*i)
    return 'Done'

In [26]:
gen = simple_gen()

In [27]:
next(gen)

simple_gen: 第0次生成并返回0


0

In [28]:
next(gen)

simple_gen: 第1次生成并返回1


1

In [29]:
next(gen)

simple_gen: 第2次生成并返回4


4

In [30]:
next(gen)

simple_gen: 第3次生成并返回9


9

总结一下，在本质上，生成器就是一种元素生成函数，它和普通函数的不同之处在于，它的返回值不是通过return返回的，而是通过yield返回的。另外一个需要注意的地方是，含有yield语句的函数中如果还配有return语句，那么这个return语句并不是用于函数正常返回的，而是StopIteration的异常说明。也就是说，生成器没有办法使用return的返回值。如果想获得该返回值，需要捕获StopIteration异常，然后输出StopIteration.value。

In [31]:
for i in gen:
    print(i)

simple_gen: 第4次生成并返回16
16
simple_gen: 第5次生成并返回25
25
simple_gen: 第6次生成并返回36
36
simple_gen: 第7次生成并返回49
49
simple_gen: 第8次生成并返回64
64
simple_gen: 第9次生成并返回81
81


In [35]:
try:
    next(gen)
except StopIteration as e:
    print(e.value)

None


### 2.2 迭代器

迭代是Python最强大的功能之一，是访问集合元素的一种方式。顾名思义，迭代器就是用于迭代操作（如for循环、while循环）的对象，它可以像列表一样迭代获取其中的每一个元素。

#### 可迭代对象

在Python中，有很多好用的数据类型，如列表、元组、字典、集合、字符串等。事实上，这些所谓的“数据类型”，更确切地说是存储数据的容器（container）。操作这些容器时，我们常需要逐个访问其中的元素。这种逐个获取容器中元素的过程，就叫“迭代”（iteration）。

简单来说，具备可迭代访问特性的对象，就叫作可迭代对象。所谓迭代器，简单来说，我们可以把它理解为能够访问容器元素的“智能指针”。

In [37]:
# 创建迭代器对象
it = iter(['A', 'B', 'C', 'D', 'E', 'F'])

while True:
    # 使用异常捕获结构
    try:
        print(next(it), end=' ')
    # 捕捉异常
    except StopIteration: 
        print('迭代器越界了！')
        break
print('OK!')

A B C D E F 迭代器越界了！
OK!


迭代器内部维护着一个状态，该状态用来记录当前迭代“指针”所在的位置，以方便下次迭代时获取正确的元素。一旦所有元素都被遍历，那么迭代器就会指向容器对象的尾部，并触发停止迭代的异常。

迭代器有一个显著的特点，那就是惰性估值（Lazy evaluation）。其含义在于，只有当迭代至某个值时，该元素才会被计算并获取。这个特性有点像“抽一鞭子走一步”的懒牛。存在即合理。这种“懒”也是有优点的，即迭代器特别适合用于遍历大文件或无限集合，因为我们不用一次性将它们全部预存到内存之中，用哪个再临时拿来即可。

#### 创建迭代器

在Python中，一切皆对象。迭代器也不例外，具体的迭代器实际上是某个迭代类定义的对象。比如list_iterator是列表类迭代器的对象，set_iterator是集合类迭代器的对象，以此类推。

所有的迭代器在设计之时通常都会在类中实现两个方法：`__iter__()`和`__next__()`。`__iter__()`方法用于返回一个迭代器对象，`__next__()`方法用于返回迭代对象内部的下一个元素值。

下面就创建一个斐波那契数列的迭代器，以直观地感受迭代器内部的执行过程：

In [38]:
class Fibonacci:
    def __init__(self):
        self.previous, self.current = 0, 1
        
    def __iter__(self):
        return self
    
    def __next__(self):
        value = self.current
        self.previous, self.current = self.current, self.current + self.previous
        return value

In [40]:
from itertools import islice
f = Fibonacci()
a = list(islice(f, 0, 10))
a

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

Python中有一个内置的模块itertools，该模块中包含了一系列用来产生不同类型迭代器的函数或类，它们都可以产生一个迭代器，然后通过for/while循环来遍历取值，当然也可以使用全局内置函数next()来取值。

islice函数的第一个参数就是一个可迭代对象，随后的参数分别是迭代对象的起始位、终止位和步长。它的用法和列表及元组的“切片”函数非常类似。事实上，islice就是“迭代分片”的意思，其中“i”表示“iterable”（可迭代对象），“slice”表示“分片”。但是，这个迭代切片不支持负数索引。

简单来说，迭代器就像一个惰性加载的工厂，等到有人需要时，它才加工产品，返回生成值，当没人搭理的时候，它就处于休眠状态，等待下一次调用来唤醒。

In [41]:
list(islice(f, 0, 10))

[89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]

## 3 异常处理

在程序编写过程中，通常有一个80/20原则，即将80%的精力花费在20%的事情上，而这20%的事情就是处理各种可能出现的错误或异常。如果想编写一个完善的高容错运行程序，且不使用异常处理机制，那么程序中将充斥着各种if语句，用于处理各种可能的意外。如果是这样的话，整个程序的结构就会变得臃肿且混乱。

而事实上，由于程序员本身存在思维盲点，即使再简单的程序，要把其中所有可能出现的错误都预想到，也是不现实的。由于无法做到“考虑完备”，Python可能会在运行时发生各类异常（Exception）。因此，一个健壮的程序，通常都要设置异常处理模块。

### 3.1 什么是异常

异常也称为例外，指的是所有可能造成计算机无法正常处理的情况，如果没有进行妥善的安排，严重的话将使计算机宕机。异常处理是一种特定的程序错误处理机制，是为了让程序员更加关注正常的程序执行序列而设计的。

In [43]:
10 * (2 / 0)

ZeroDivisionError: division by zero

In [44]:
4 + num * 2

NameError: name 'num' is not defined

In [45]:
'4' + 4

TypeError: can only concatenate str (not "int") to str

In [46]:
f = open('abc.txt', 'r')

FileNotFoundError: [Errno 2] No such file or directory: 'abc.txt'

有了异常，就应该有相应的异常处理手段，这样才能确保这些异常不会导致数据丢失或系统运行遭到破坏等灾难性后。

### 3.2 异常处理三步走

在高级语言（如C++、Java）中，通常都内置了一套“try...except...finally...”的三步走式错误处理机制，Python也不例外。如果我们认为某个代码块可能会出错，就可以用try来“管辖”这段代码。一旦发生异常，则不会继续执行后续代码，而是直接跳转至异常处理代码（即except语句块），执行except。如果try代码块没有异常发生，则忽略except子句。一个try代码块中可能包含多个except子句，可分别处理不同类型的异常，但最多只有一个分支会被执行。

异常处理过程中如果还有finally语句块，则还需执行finally语句块。finally语句块为可选项，非必需，一旦设置，无论是否发生异常，就如其名称所彰显的含义一般，程序“最终”都要在finally语句块上“走一遭”。

In [47]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print('发生异常：', err)
finally:
    print('这是可选的输出！')
print('正常输出！')

发生异常： division by zero
这是可选的输出！
正常输出！


凡是涉及管辖范围的，管辖者（如这里的try和except）都要以冒号（:）来彰显自己的“势力范围”，而被管辖者则要以统一的缩进来表示服从。

如果没有异常处理，我们的Python程序是脆弱的，稍有风吹草动，整个程序都会受到牵连，随后的程序均无以为继，进而停止运行，这对大型程序而言是难以承受之重。所以说，异常处理能大大提升整个程序的“鲁棒性”（Robust，也称健壮性）。

还会有这样的需求：系统给定的异常处理类型不够用，能否设计并抛出个性化的异常呢？答案是，当然可以。这时我们可以利用raise（举起）来抛出一个自定义的异常。

In [51]:
def sqrt(n):
    if n < 0:
        raise Exception('参数n={}不能为负,不能开平方!'.format(n))
    return n ** 0.5

In [53]:
try:
    sqrt(-5)
except Exception as e:
    print('ERROR:', e)

ERROR: 参数n=-5不能为负,不能开平方!


## 4. 错误调试

异常处理模块能帮助我们在运行期间处理异常信息，但Python代码还有更为基础的错误—语法错误和逻辑错误。语法错误相对简单，在解释器的帮助下，我们很快就能定位错误所在。但对逻辑错误的调试就难多了，几乎“引无数coder竞折腰”！

这些语法或逻辑层面的错误，构成了各式各样的代码bug（代码缺陷）。为了调试错误，我们需要知道，出错时哪些变量的值是正确的，哪些变量的值是错误的。因此，我们需要掌握一些代码调试的基本技巧。

### 4.1 利用print()输出观察变量

第一种方法，简单而有效，直接而粗暴，就是用print()把需要观察的变量打印出来。如果使用jupyter交互式编程，可以直接输出变量的值，很容易就能发现错误。

In [54]:
def foo(s):
    # 字符串转换为整型
    n = int(s)
    # 输出观察变量n的值
    print('n = {}'.format(n))
    return 10 / n

In [55]:
foo('0')

n = 0


ZeroDivisionError: division by zero

根据打印处的信息和错误信息（division by zero），可以很容易地定位错误所在：代码第6行，作为分母，n值为0。

### 4.2 assert断言

用print()观察变量的不足之处在于，调试完毕后，我们还得手动将它们删掉，如果调试工作量较大，造成print()满天飞，删除大量print()语句的工作量也不容小觑。而且，如果程序中到处充斥着print()语句，输出信息也会非常繁杂，给程序员造成困扰。

因此，就有了第二种方法—断言（assert）。凡是可用print()来辅助查看的，都可以用assert来替代。它用来测试某个条件（condition）的布尔值，系统默认这个条件为真，此时断言悄然无息，我们感知不到它的存在。但是，一旦条件为假，就会触发异常。

In [56]:
def average(scores):
    assert len(scores) != 0
    return sum(scores) / len(scores)

In [57]:
average([])

AssertionError: 

In [58]:
average([90, 85, 78, 82])

83.75

使用assert的好处在于，当判断条件为真时，用户是感觉不到assert的，因为assert只有当判断条件为假时才“刷存在感”，给出错误信息。错误信息一旦给出，在某种程度上就定位了代码的bug所在，从而达到了程序调试的目的。调试完毕后，用户无须删除assert语句.

当开发的项目规模比较大时，我们会发现，logging才是终极武器。logging是Python的日志模块。使用这个模块的好处在于，它允许我们指定记录信息的级别，有debug、info、warning、error等。我们可以根据需要输出不同级别的信息。

例如，当指定level=INFO时，logging.debug就不起作用了。同理，指定level=WARNING后，debug和info就不起作用了。这样一来，我们就不必担心太多输出信息会冲淡关注力。关于这个模块的知识，就留给“爱折腾”的“童鞋们”自学吧。

**高手，永远都是自学出来的**！